In [2]:
from tqdm import tqdm
import pandas as pd
import numpy as np

In [4]:
# load data
data = pd.read_csv('/Users/iseunghyeon/Desktop/[프로젝트정리]/resource/VODs_1223_0003.csv')
temp_data = data[['title','SMRY']][:3]
temp_data

,title,SMRY
0,빅매치(9월 이벤트),"흥행 끝판왕 이정재의 2014년 선택. 액션은 특급, 질주는 본능, 유머는 옵션. ..."
1,비정규직 특수요원,대한민국 최고기관들이 보이스피싱에 탈탈 털렸다. 만년 알바 인생 장영실은 35살의 ...
2,비밀의 집,본 회차는 방송사의 사정으로 줄거리를 제공하지 않습니다.


In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser
from langchain.llms import OpenAI


In [13]:
# open ai의 api key를 발급 받고, 발급 받은 api key를 문자열로 

open_ai_key = "{api_key}"
llm = OpenAI(openai_api_key=open_ai_key)

In [ ]:
# GPT가 생성한 단어들을 전처리하기 위한 clean_word 함수 작성.
    # '-' 문자를 제외한 나머지 특수문자는 삭제하고, 단어는 소문자로 통일

def clean_word(text: str) -> str:
    import re
    pattern = re.compile(r'[^A-Za-z-]')
    cleaned_text = pattern.sub('', text)
    cleaned_text_lower = cleaned_text.lower()
    return cleaned_text_lower

In [15]:
# OutputParser를 통해 GPT에서 생성한 데이터를 형식에 맞춰 추출하도록 한다.

class CommaSeparatedListOutputParser(BaseOutputParser):
    """LLM 아웃풋에 있는 ','를 분리해서 리턴하는 파서."""
    def parse(self, text: str):
        return text.strip().split(", ")


In [16]:
# LLM Chain을 통해 데이터  추출.

def run_llm_chain(description: str, llm_openai_api_key:str, template:str):
    
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)

    # human_template: 우리 프로젝트에서는 사용하진않음.
    human_template = "{text}"
    human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

    chain = LLMChain(
        llm=ChatOpenAI(openai_api_key=llm_openai_api_key),
        prompt=chat_prompt,
        output_parser=CommaSeparatedListOutputParser()
    )

    results = chain.run(description)

    # GPT가 생성한 단어 중 문자가 짧거나 긴 단어는 필터링 하기
    
    results = [clean_word(result) for result in results if 2 <= len(clean_word(result)) <= 20]
    
    return results




In [17]:
# Lang Chain에 추출할 Prompt를 제시하기 

templates = {

    # 줄거리를 통해 무드 장르를 추출하는 템플릿 
    'template_A': """
    You are now an AI that helps film critics express themselves richly.
    Please provide the plot of the movie or drama.
    List 5 mood words in English that capture the essence of the story.
    For example, mood words include “interesting,” “mysterious,” “charming,” and “sense of urgency.”

    Separate each mood word with a comma (,) and refrain from adding any additional information.
""",

    # 줄거리를 통해 세부 장르를 추출하는 템플릿
'template_B':"""
    You are now an AI that helps film critics express themselves richly.
    Please provide the plot of the movie or drama.
    List 5 genre words in English that capture the essence of the story.
    For example, genre words include “joker,” “mysterious,” "chasing",“car chasing,”,"history"and “war.”

    Separate each word with a comma (,) and refrain from adding any additional information.
""",

    # 줄거리를 통해 주제어 단어를 추출하는 템플릿
'template_C':"""
    You are now an AI that helps film critics express themselves richly.
    Please provide the plot of the movie or drama.
    List 5 subject words in English that capture the essence of the story.
    For example, subject words include “joker,” “cat,” "couple",“car”,"history"and “war.”

    Separate each word with a comma (,) and refrain from adding any additional information.

"""
}

In [19]:
# llm_openai_api_key의 인자로 앞서 명시한 api key 대입

llm_openai_api_key = open_ai_key

tqdm.pandas()

# template_value 에서 3개의 템플릿들을 돌면서 메타 데이터를 추출함.
for template_key, template_value in templates.items():
    temp_data[template_key] = temp_data['SMRY'].progress_apply(

         # run_llm_chain의 함수인자로, description: 줄거리, api key, template_value: 구성한 템플릿들을 주어, 메타 데이터 추출하기.
        lambda description: [word for word in run_llm_chain(description, llm_openai_api_key, template_value)]
    )

100%|██████████| 3/3 [00:04<00:00,  1.41s/it]


In [20]:
temp_data

,title,SMRY,template_A,template_B,template_C
0,빅매치(9월 이벤트),"흥행 끝판왕 이정재의 2014년 선택. 액션은 특급, 질주는 본능, 유머는 옵션. ...","[action-packed, thrilling, humorous, intense, ...","[action, thriller, comedy, kidnapping, racing]","[action, kidnapping, fighter, race, match]"
1,비정규직 특수요원,대한민국 최고기관들이 보이스피싱에 탈탈 털렸다. 만년 알바 인생 장영실은 35살의 ...,"[mysterious, thrilling, suspenseful, intense, ...","[voicephishing, governmentagency, temporaryemp...","[voicephishing, temporaryemployment, nationals..."
2,비밀의 집,본 회차는 방송사의 사정으로 줄거리를 제공하지 않습니다.,"[mysterious, intense, suspenseful, emotional, ...",[sorry],[sorry]
